<a href="https://colab.research.google.com/github/roccaab/WaveletGAN/blob/main/DownloadData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import csv
import os
import glob
import subprocess
import zipfile
import shutil
from zipfile import ZipFile, ZIP_DEFLATED

def scarica_e_estrai_file(esm_event_id, station_code, zip_output="output.zip"):
    """
    Esegue la chiamata curl per scaricare il file zip relativo all'evento/stazione
    e lo estrae in una cartella temporanea.
    Restituisce il percorso della cartella temporanea dove sono stati estratti i file.
    """
    url = (
        "https://itaca.mi.ingv.it/itaca40ws/eventdata/1/query"
        f"?eventid={esm_event_id}"
        f"&station={station_code}"
        "&format=ascii"
    )
    print(url) # Stampa per debug del risultato della CURL

    curl_cmd = f'curl -s -X GET "{url}" -o {zip_output}'
    subprocess.run(curl_cmd, shell=True, check=True)

    temp_dir = f"temp_{esm_event_id}"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir, exist_ok=True)

    with zipfile.ZipFile(zip_output, 'r') as zf:
        zf.extractall(temp_dir)

    return temp_dir

def filtra_e_salva_file_ascii(orig_file, out_file_name):
    """
    Legge il file ASCII (orig_file) ed elimina tutte le righe:
      - vuote
      - che iniziano con '#'
      - che contengono ':'
      - che non sono parse come numero float
    Vengono scritti solo i valori numerici (una colonna).
    """
    with open(orig_file, 'r') as fin, open(out_file_name, 'w') as fout:
        for line in fin:
            line = line.strip()
            # Salta righe vuote o righe che iniziano con '#' o che contengono ':'
            if not line or line.startswith('#') or ':' in line:
                continue
            # Prova a interpretare la linea come float (es. 0.123, -0.35, 1.23E-03, ecc.)
            try:
                float(line)
                fout.write(line + "\n")
            except ValueError:
                # Se non è convertibile in float, la scartiamo
                continue

def main():
    csv_input = "ID_events.csv"   # Il file con i dati di input
    zip_output = "output.zip"     # Nome temporaneo per ogni zip scaricato

    out_file_counter = 1
    real_files_generated = []  # Per tenere traccia di tutti i RealX.csv prodotti

    with open(csv_input, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader, None)  # Salta l'header del CSV se presente

        for row in reader:
            # Estraggo gli ID
            esm_event_id = row[0].strip()
            station_code = row[7].strip()

            print(f"=== Elaborazione evento {esm_event_id}, stazione {station_code} ===")

            # Scarico e estraggo lo zip
            try:
                temp_dir = scarica_e_estrai_file(esm_event_id, station_code, zip_output)
            except Exception as e:
                print(f"Impossibile scaricare/estrarre lo zip per {esm_event_id}: {e}")
                continue

            # Recupero TUTTI i file estratti
            all_extracted_files = glob.glob(os.path.join(temp_dir, '**'), recursive=True)

            # Filtra i file che contengono "BHE" o "BHN" nel nome
            file_list = []
            for path in all_extracted_files:
                if os.path.isfile(path):
                    filename = os.path.basename(path)
                    # Se il filename ha "BHE" o "BHN" in mezzo
                    if 'HGE' in filename or 'HGN' in filename:
                        file_list.append(path)

            if not file_list:
                print(f"  Nessun file con componente BHE/BHN trovato per {esm_event_id} - {station_code}")
            else:
                for ascii_file in file_list:
                    out_file_name = f"Real{out_file_counter}.csv"
                    try:
                        filtra_e_salva_file_ascii(ascii_file, out_file_name)
                        real_files_generated.append(out_file_name)
                        print(f"  File {ascii_file} salvato come {out_file_name}")
                        out_file_counter += 1
                    except Exception as e:
                        print(f"  Errore durante l'elaborazione di {ascii_file}: {e}")

            # Pulizia temporanei di questa iterazione
            shutil.rmtree(temp_dir, ignore_errors=True)
            if os.path.exists(zip_output):
                os.remove(zip_output)

    # Al termine, comprimo tutti i RealX.csv in Real.zip e poi li elimino
    if real_files_generated:
        with ZipFile("Real.zip", "w", compression=ZIP_DEFLATED) as zf:
            for rf in real_files_generated:
                zf.write(rf)
                os.remove(rf)
        print(f"\nCreato Real.zip con {len(real_files_generated)} file CSV e rimossi i singoli RealX.csv")
    else:
        print("\nNessun file RealX.csv generato, non viene creato alcuno zip.")

    print("Elaborazione completata!")

if __name__ == "__main__":
    main()


=== Elaborazione evento EMSC-20160824_0000006, stazione AQK ===
https://itaca.mi.ingv.it/itaca40ws/eventdata/1/query?eventid=EMSC-20160824_0000006&station=AQK&format=ascii
  File temp_EMSC-20160824_0000006/IT.AQK.00.HGN.D.EMSC-20160824_0000006.ACC.MP.ASC salvato come Real1.csv
  File temp_EMSC-20160824_0000006/IT.AQK.00.HGE.D.EMSC-20160824_0000006.ACC.MP.ASC salvato come Real2.csv
=== Elaborazione evento EMSC-20160824_0000007, stazione AQK ===
https://itaca.mi.ingv.it/itaca40ws/eventdata/1/query?eventid=EMSC-20160824_0000007&station=AQK&format=ascii
  File temp_EMSC-20160824_0000007/IT.AQK.00.HGE.D.EMSC-20160824_0000007.ACC.MP.ASC salvato come Real3.csv
  File temp_EMSC-20160824_0000007/IT.AQK.00.HGN.D.EMSC-20160824_0000007.ACC.MP.ASC salvato come Real4.csv
=== Elaborazione evento EMSC-20160824_0000013, stazione AQK ===
https://itaca.mi.ingv.it/itaca40ws/eventdata/1/query?eventid=EMSC-20160824_0000013&station=AQK&format=ascii
  File temp_EMSC-20160824_0000013/IT.AQK.00.HGN.D.EMSC-20160